In [1]:
# spark imports 
import pyspark
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import math
from pyspark.ml.feature import Bucketizer
!pip install findspark

import findspark
import pandas as pd
from collections import Counter


pd.__version__

findspark.init()
findspark.find()

    
spark = ( 
    SparkSession
    .builder
    .master("local[20]")
    .appName("AssociationRuleMining")
    .config("spark.driver.cores", '5')
    .config("spark.driver.memory", '20g')
    .config("spark.executor.memory", '10g')
    .config("spark.executor.cores", '2')
    .config("spark.dynamicAllocation.shuffleTracking.enabled", "true")
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.dynamicAllocation.minExecutors", 6)
    .config("spark.dynamicAllocation.maxExecutors", 18)
    .config("spark.executor.instances", '12')
    .config("spark.sql.shuffle.partitions",'12')
    .config("spark.submit.deployMode", "client")
    .getOrCreate()
    )


# Sanity check:  print spark configuration data.
for a in spark.sparkContext._conf.getAll():
    print(a)
#File name to Binned Data
path_to_data = '100BenignAndDiscoveryWithFullAttributes.csv'

#reading data into pandas dataframe
store_data = pd.read_csv(path_to_data)


('spark.driver.extraJavaOptions', '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED')
('spark.dynamicAllocation.minExecutors', '6')
('spark.executor.memory', '10g')
('spark.app.startTime', '1677710866414')
('spark.executor.instances', '12')
('spark.driver.cores', '5')
('spark.executor.id', 'driver')
('spark.dynamicAllocat

In [2]:
store_data.head(5)

,label_tactic,dest_ip_zeek,src_ip_zeek,proto,conn_state,local_orig,local_resp,history,service,duration,orig_bytes,orig_pkts,orig_ip_bytes,resp_bytes,resp_pkts,resp_ip_bytes,missed_bytes
0,0,2,2,1,1,1,1,1,3,0,0,2,2,0,2,2,1
1,0,0,0,1,1,1,1,1,3,0,0,2,3,0,2,2,1
2,0,2,2,1,1,1,1,1,3,0,0,2,2,0,2,2,1
3,0,2,2,1,1,1,1,1,3,0,0,2,2,0,2,2,1
4,0,2,2,1,1,1,1,1,3,0,0,2,3,0,2,2,1


In [5]:
# The first step in ARM - Fiding all the possible unique values within the dataset to (later) find 
# all the unique unions that are possible to test against the chosen thresholds 

def findUniqueNum():     
    #Contains all the unique values within the dataframe 
    numBin = []
    
    for i in range(len(transactions)):
        for x in transactions[i]:
            if (x not in numBin): 
                numBin.append(x)
                
    # Can test this on a smaller dataset for a better understanding          
    #print(type(numBin)) 
    #print('The original Unique Numbers' + str(numBin))
    
    return numBin

In [6]:
# Method focuses on finding the first frequet unique individuals within dataset that passes support thresholds. 
# Only called once. 
# Will use this itemset to combine (later) into larger itemsets 
def firstFreqItemSet(numBin): 
    
    associationRules = []  #this stays empty but is later used in Union()
    newNumBin = [] # contains all unique individuals that had a high enough support value 
    ruleNum = 0 # to be used to call Union method

    for i in numBin:
        # and every transaction 
        for x in range(len(transactions)):  
            # for every unique value found in a transaction 
            if(i in transactions[x]):    
                #add a counter value 
                c[i]+=1
    
    for i in numBin:
        if calculateSupport(c[i]) == True:
            l[frozenset([i])]+=c[i]
            newNumBin.append(i) 
            print(str(i))   # to be used for testing purposes on a smaller dataset 
            
    union(newNumBin, l, associationRules, ruleNum)        
    

In [7]:
# Where num = the count of times that the itemset occurs 
# calculates support to decide if support is above the chosen threshold
# returns true if it is above chosen threshold 
def calculateSupport(num):
    support = num/len(transactions)
    if (support >= sp):
        return True 
    else:
        return False 

In [8]:
# Works to combine frequet itemsets 
# Tests against set thresholds 

def union(numBin, l, associationRules, ruleNum):
    c = Counter() #resests C 
    nc = set() # holds sets 
    temp = list(l) # turns counter passed into method into a set 
    
    #Unions all possible values within temp 
    #adds to nc 
    for i in range(0,len(temp)):
        for j in range(i+1,len(temp)):
            t = temp[i].union(temp[j])
            nc.add(t)

    nc = list(nc) #converts all possible unions to a list 
    cList = [] 
    
    #for every possible union
    for i in nc:
        y = list(i)        
        #if confidence is deemed high enough 
        if(calculateConfidence(y) == True):
            c[i] = 0
            #and you can find it in a transaction 
            for q in range(len(transactions)):
                
                # turn the union into a set again and check for subset of set of one transaction
                t = transactions[q]
          
                temp = set(t)           
                tempT =set(i)
            
                if(tempT.issubset(temp)):
                        # then increment the counter 
                        c[i]+=1

    l = Counter()
    
    #For every set with high enough confidence values, check the support 
    for i in c:
        if calculateSupport(c[i]) == True:
            l[i]+=c[i]
            
    #Create rule         
    for i in l:
    
        support = (l[i]/len(transactions))
        #print('This is the support 1  ' + str(support))
        #print(str(i))
        confidence = returnConfidence(i)
        lift = returnLift(i, confidence)
        ruleNum = ruleNum + 1
        rule = ('Rule: ' + str(ruleNum) + ' ' + str(list(i)) + '[ support =' + str(support) + '  confidence = ' + str(confidence) + ' lift = ' + str(lift) + ' ]' + '\n') 
        
        # Seeing if Rule is already within the list of rules, just a double check  
        associationRules = repeatIdentifier(rule, associationRules)  
    
    #If no more unions pass the thresholds end program
    if(len(l) == 0):
        printResults(associationRules) 
    #else continue on with the rest of the unions 
    else:       
        #print('size of list of acceptable unions' + str(len(numBin))) // Testing purposes
        union( numBin, l, associationRules, ruleNum) 

In [9]:
#Finds if rule is already contained in list. Adds it to list if the rule is not found and then returns the list
#Works to prevent repeat association rules 
def repeatIdentifier(rule, ruleList): 
    if ((rule in ruleList) == True):
        print('Rule already in List')
    else:
        ruleList.append(rule)

    return ruleList
    

In [10]:
# calculates confidence 
def calculateConfidence(num):
    numList = list(num)
    numCountAntecedent = []
    
    #Contains all the unique values within the dataframe 
    numBin = []
    numCountA = 0
    numCountP = 0
    
    for x in range(len(transactions)): 
        x = list(transactions[x]) 
        if (x.count(num[0]) > 0):
            numCountA = numCountA + 1        
    
    #The whole transaction
    for x in range(len(transactions)):
        setX = set(transactions[x])
        num = set(num) 
        if(num.issubset(setX)):
            numCountP = numCountP + 1
    confidence = numCountP/numCountA
    if (confidence >= conf):
        if (returnLift(num, confidence) >=liftE):
            return True 
        else:
            return False
    else: 
        return False 

In [11]:
#Returns the Confidence of an Association Rule 
#Confidence=transactions containing A and B / transactions containing A 

def returnConfidence(num): 
    
    numList = list(num)
    numCountAntecedent = []
    numBin = []
    numCountA = 0
    numCountP = 0
    
    # Number of transactions that the antecedent is available 
    for x in range(len(transactions)): 
        x = list(transactions[x]) 
        if (x.count(numList[0]) > 0):
            numCountA = numCountA + 1        
    
    #Number of transactions that the consequent and the antecedent is available 
    for x in range(len(transactions)):
        setX = set(transactions[x])
        num = set(num) 
        if(num.issubset(setX)):
            numCountP = numCountP + 1    
    confidence = numCountP/numCountA
    return confidence

In [12]:
# Method Calculates the Lift value and returns it 
# Lift = Confidence of if A then B / support of B

def returnLift(transaction, confidence):
    
    numList = list(transaction)
    
    numCountAntecedent = []
    numBin = []
    newWord =''
    numCountA = 0
    numCountP = 0
    
    #Number of occurences of the antecendet 
    for x in range(len(transactions)): 
        x = list(transactions[x]) 
        if (x.count(numList[0]) > 0):
            numCountA = numCountA + 1   
            
    numList.remove(numList[0])
    
    #Number of occurences of the consequent 
    for x in range(len(transactions)):
        setX = set(transactions[x])
        num = set(numList) 
        if(num.issubset(setX)):
            numCountP = numCountP + 1
            
    supportP = numCountP/len(transactions)
    
    lift = confidence/supportP
    
    return lift

In [13]:
#Prints association rules to file 
def printResults(associationRules):     
    file = open('Test#.txt', 'w')
    file.writelines(associationRules)   
    print('End of program')

In [14]:
#Altering Binned Data information to produce a new list with header in each data point 
transactions = store_data.values.tolist()

columnNames = []
newList = [] 
bigList = []

# iterating the columns
for col in store_data.columns:
    columnNames.append(col) 
    
for i in range(len(transactions)):
    z = -1
    newList = [] 

    for x in transactions[i]:
        T = str(x)
        z = z + 1
        d = str(columnNames[z]) + '=' + T
        newList.append(d) 
    bigList.append(newList)
transactions = bigList

In [15]:
transactionsWHeader = []
indicator = 0


#indicating ARM Thresholds 
liftE = 0 #lift
sp = 0.99 #Support
conf =0.99 #Confidence

a = Counter()
b = []


#Create global counter variables 
c = Counter()
l = Counter()
l2 = Counter()

emptyList = []
supportValues = [] 
num = findUniqueNum()
firstFreqItemSet(num) 

In the findUniqueNum() method, about to traverse the dataframe
src_ip_zeek=2
local_orig=1
local_resp=1
missed_bytes=1
End of program
